# Systemy Inteligentne w biznesie
## Laboratorium: Czyszczenie danych i przygotowanie zbioru do analizy
## _autor: Mariusz Krzyżowski_



## 1. Wprowadzenie
Każdy, kto choć trochę miał wspólnego z analizą danych, ma świadomość, że dane przekazane do analizy rzadko kiedy przychodzą w postaci łatwej i przyjemnej do ich obróbki: bardzo często pojawiają się w nich wartości brakujące, puste czy wręcz błędne (np. pod względem typu danych). Osoby gromadzące dane mogą niektórych z nich nie posiadać, innych przez pomyłkę nie przekazać, a przy wprowadzaniu kolejnych wręcz się pomylić (np. przesuwając przypadkowo przecinek lub wykonując literówki). Uważa się, że wstępne przygotowanie danych oraz usunięcie wszelkich niedoskonałości tak, aby analiza danych była skuteczna i dostarczała odpowiednich danych, może zająć nawet do 80% czasu pracy nad zbiorem.  
Celem laboratorium jest wykonanie czyszczenia danych znajdujących się w przekazanym zbiorze oraz przygotowanie ich do analizy, z wykorzystaniem ich modyfikacji, ujednolicania czy rozdzielania. W tym celu zostanie wykorzystana biblioteka `pandas` języka Python.  
Wykonana praca nad zbiorem składa się z kilku etapów, każdy z nich został umieszczony w oddzielnym rozdziale.

## 2. Załadowanie danych
Na wstępie importowane są niezbędne biblioteki: `pandas` oraz `numpy` umożliwiające pracę nad zbiorami danych. Następnie przekazane dane zostają wczytane. Warto zwrócić uwagę na dodatkowe parametry:
* `sep = "\t"` oznacza, że seperatorem w pliku jest tabulator
* `index_col="PassengerId"` oznacza, że kolumną indeksującą jest PassengerId - nie jest wówczas tworzona dodatkowa kolumna z indeksem

In [275]:
import pandas as pd
import numpy as np

titanic = pd.read_csv("TitanicMess.tsv", sep="\t", index_col="PassengerId")

## 3. Wyświetlenie próbki danych
Aby mieć świadomość, z jakimi danymi mamy do czynienia, wyświetlonych zostanie pierwszych dziesięć rekordów znajdujących się w zbiorze.

In [276]:
titanic.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,"8,4583",NaN,Q,Titanic
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,"51,8625",E46,S,Titanic
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,"21,075",NaN,S,Titanic
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,"11,1333",NaN,S,Titanic


## 4. Wyświetlenie podstawowych informacji o danych w celu poznania ich charakteru oraz diagnozy problemów do rozwiązania podczas czyszczenia
Praca ze zbiorem danych została rozpoczęta od kilku wstępnych kroków, dzięki którym zapoznano się m.in. z ich liczbą, typami danych, liczbą brakujących wartości.

### a. wyświetlenie liczby wierszy i kolumn

In [277]:
titanic.shape

(892, 12)

### b. wyświetlenie nazw kolumn oraz liczby pustych wartości w kolumnach

In [278]:
titanic.columns.tolist()

['Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked',
 'ship']

In [279]:
titanic.isnull().count()

Survived    892
Pclass      892
Name        892
Sex         892
Age         892
SibSp       892
Parch       892
Ticket      892
Fare        892
Cabin       892
Embarked    892
ship        892
dtype: int64

### c. wyświetlenie podstawowych informacji o danych

In [280]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892 entries, 1 to 1000
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Survived  892 non-null    int64 
 1   Pclass    892 non-null    int64 
 2   Name      892 non-null    object
 3   Sex       892 non-null    object
 4   Age       719 non-null    object
 5   SibSp     892 non-null    int64 
 6   Parch     892 non-null    int64 
 7   Ticket    892 non-null    object
 8   Fare      892 non-null    object
 9   Cabin     207 non-null    object
 10  Embarked  890 non-null    object
 11  ship      892 non-null    object
dtypes: int64(4), object(8)
memory usage: 90.6+ KB


Analizując powyższą tabelę oraz wyniki z podpunktów a i b można wyciągnąć następujące ogólne wnioski, które w dalszej części pracy będą potwierdzane lub realizowane poprzez modyfikację i czyszczenie danych:
* Kolumna `Survived` zawiera wartości liczbowe i nie ma brakujących wartości - można podejrzewać, że kolumna ta powinna zawierać 2 wartości:
    * 0 - oznaczająca śmierć pasażera
    * 1 - oznaczająca przeżycie pasażera
* Kolumna `PClass` zawiera liczby całkowite i nie ma brakujących wartości - można podejrzewać, że będzie docelowo zawierała wartości 1, 2 i 3 - oznaczające numer klasy
* Kolumna `Sex` nie ma brakujących wartości - należy doprowadzić do sytuacji, w której będzie zawierała wartości "female" oraz "male"
* W przypadku kolumny `Embarked` brakuje 2 wartości, kolumny `Age` - 173 wartości, kolumny `Cabin` - aż 685 wartości, co stanowi blisko 80% brakujących danych w kolumnie. Pozostałe kolumny mają uzupełnione wszystkie wartości.
* Kolumny `Age` oraz `Fare` zostały niepoprawnie odczytane jako ogólny typ `object` - prawdopodobnie z powodu błędów we wprowadzonych danych nie wszystkie wartości zostały poprawnie zinterpretowane jako liczby
* Kolumna `ship` zawiera 892 informacje o statku - ze względu na charakter analizowanego zbioru prawdopodobnie zawiera jedną unikalną wartość `Titanic`

## 5. Analiza kolumny `ship`
Pierwszą szczegółowo analizowaną i modyfikowaną kolumną jest kolumna `ship` oznaczająca nazwę statku. Wiemy już, że nie posiada ona brakujących wartości. Przeanalizujmy, jakie wartości w niej występują:


In [281]:
titanic["ship"].unique()

array(['Titanic'], dtype=object)

Ze zwróconej tablicy wynika, że w kolumnie znajduje się tylko jedna wartość - Titanic. Jest to uzasadnione faktem, że zbiór danych dotyczy pasażerów jednego statku - Titanica. W związku z tym kolumna ta nie będzie użyteczna w dalszych rozważaniach i analizie danych - można ją usunąć:

In [282]:
titanicOutput = titanic.drop(columns="ship")
titanicOutput.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,"8,4583",NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,"51,8625",E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,"21,075",NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,"11,1333",NaN,S


## 6. Analiza kolumny `Survived`
Kolejną analizowaną kolumną jest kolumna `Survived`. Należy się przekonać, czy początkowe założenie, że w kolumnie tej znajdują się wyłącznie wartości 0 (oznaczające śmierć pasażera) oraz 1 (oznaczające przeżycie pasażera). Wyświetlmy unikalne wartości tej kolumny:

In [283]:
titanic["Survived"].unique()

array([0, 1], dtype=int64)

Początkowe założenie jest prawdziwe - kolumna jest całkowicie wypełniona poprawnymi wartościami - w związku z tym żadna ingerencja ani modyfikacje danych nie są wskazane.

## 7. Analiza kolumny `Pclass`
Następnym krokiem analizy jest przejrzenie wartości kolumny `Pclass`. Analiza polega na sprawdzeniu, czy początkowe założenie, że w kolumnie tej znajdują się wyłącznie wartości 1, 2 oraz 3, które wskazują na numer klasy. Wyświetlmy unikalne wartości tej kolumny:

In [284]:
titanic["Pclass"].unique()

array([3, 1, 2], dtype=int64)

Wniosek był poprawny, zatem zawartość kolumny nie musi być modyfikowana.

## 8. Analiza kolumny `Cabin`
Następną analizowaną kolumną jest kolumna `Cabin`. Na wstępie przypomnijmy procent pustych wartości w tej kolumnie:

In [285]:
titanic["Cabin"].isnull().sum() / titanic.shape[0]

0.7679372197309418

Brakuje blisko 80% wartości w analizowanej kolumnie, co może uniemożliwić jakiekolwiek działanie związane z tą kolumną. Wyświetlmy jednak jeszcze unikalne wartości kolumny `Cabin`:

In [286]:
titanic["Cabin"].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'C23 C25 C27',
       'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31',
       'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69', 'D47',
       'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32', 'B4',
       'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87', 'B77',
       'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49', 'D',
       'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66', 'C7',
       'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128', 'D37',
       'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44', 'A34', 'C104',
       'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14', 'B37', 'C30',
       'D20', 'B79', 'E25', 'D46', 'C95', 'B38', 'B39', 'B22', 'C86',
       'C70', 'A16', 'C101', 'C68', 'A10', 'E68', 'B41', 'A20', 'D19',
       'D50', 'D9', 'A23', 'B50', 'A26', 'D48', 'E58', 'C126', 'B71',
       'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63', 'C62 C64', 'E24',
       'C90'

Ze względu na fakt, że istnieje dużo różnych wartości w tej kolumnie, a mimo to brakuje zdecydowanej większości danych, postanowiono usunąć tę kolumnę ze zbioru danych. Nie znalazłaby ona zastosowania w jakiejkolwiek dalszej analizie.

In [287]:
titanicOutput = titanicOutput.drop(columns="Cabin")
titanicOutput.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,"8,4583",Q
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,"51,8625",S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,"21,075",S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,"11,1333",S


## 9. Analiza kolumny `Sex`
Zgodnie z początkową analizą tej kolumny, powinny znajdować się w niej wartości wyłącznie `male` oraz `female`. Sprawdźmy zatem, czy tak jest:

In [288]:
titanic["Sex"].unique()

array(['male', 'female', 'malef', 'mal', 'fem', 'femmale'], dtype=object)

Okazało się, że poza przewidywanymi wartościami w kolumnie znajdują się również 4 inne. Ich analiza pozwala przypuszczać, że powstały one w wyniku literówek lub skrótów stosowanych przez osoby wprowadzające dane. W związku z tym w celu przygotowania kolumny do analizy poprawiono błędne wartości ich poprawnymi odpowiednikami (`malef`, `mal` poprawiono na `male`; `fem` oraz `femmale` zmieniono na `female`)

In [289]:
titanicOutput["Sex"] = titanicOutput["Sex"].replace(["malef","mal"],"male")
titanicOutput["Sex"] = titanicOutput["Sex"].replace(["fem","femmale"],"female")
titanicOutput["Sex"].unique()

array(['male', 'female'], dtype=object)

Po wykonaniu powyższych instrukcji wszystkie dane w kolumnie `Sex` są już poprawne, co kończy analizę tej kolumny.

## 10. Analiza kolumny `Embarked`
Analizę kolumny `Embarked` rozpoczęto od przypomnienia liczby brakujących wartości oraz wyświetlenia unikalnych wartości w ramach tej kolumny

In [290]:
titanic["Embarked"].isnull().sum()

2

In [291]:
titanic["Embarked"].unique()

array(['S', 'C', 'Q', 'So', nan, 'Co', 'Qe'], dtype=object)

Po analizie zwróconych danych można podejrzewać, że wartości `So`, `Co` oraz `Qe` powstały w wyniku pomyłki. Można się o tym przekonać sprawdzając, jak często one występowały:

In [292]:
titanic["Embarked"].value_counts()

S     643
C     167
Q      76
So      2
Co      1
Qe      1
Name: Embarked, dtype: int64

Powyższa teza została potwierdzona, w związku z tym można dokonać odpowiednich poprawek (`So` zostanie zmienione na `S`, `Co` na `C`, `Qe` na `Q`).

In [293]:
titanicOutput["Embarked"] = titanicOutput["Embarked"].replace("So", "S")
titanicOutput["Embarked"] = titanicOutput["Embarked"].replace("Co", "C")
titanicOutput["Embarked"] = titanicOutput["Embarked"].replace("Qe", "Q")
titanicOutput["Embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)

Ze względu na fakt, że brakuje zaledwie dwóch wartości, można podjąć próbę ich wypełnienia. Ze względu na typ wartości (łańcuchy znaków) nie można wykorzystać matematycznego sposobu uzupełnienia wartości, ale biorąc pod uwagę fakt, że brakuje ok. 0.002% wartości uzupełnienie ich wartościami losowymi nie będzie miało znaczenia dla analizy.

In [294]:
import random
embarkedValues = [x for x in titanicOutput["Embarked"].unique() if pd.notnull(x)]
titanicOutput["Embarked"] = titanicOutput["Embarked"].fillna(random.choice(embarkedValues))

Usunięto pomyłki oraz wypełniono brakujące wartości, na czym kończy się analiza omawianej kolumny.

## 11. Analiza kolumny `Age`
Kolumna `Age` została odczytana jako typ `Object`, co jest niepokojącą oznaką. Przeanalizujmy wartości znajdujące się w tej kolumnie i postarajmy się doprowadzić do sytuacji, w której będzie możliwe zmienienie typu wartości na typ przechowujący liczby rzeczywiste (`float64`).

In [295]:
titanic["Age"].unique()

array(['22', '38', '26', '35', nan, '54', '2', '27', '14', '4', '58',
       '20', '55', '31', '34', '15', '8', '19', '40', '.9', '66', '28',
       '42', '21', '18', '3', '7', '49', '29', '65', '28,5', '5', '11',
       '45', '17', '32', '16', '25', '0,83', '30', '33', '23', '24', '46',
       '59', '71', '37', '47', '14,5', '70,5', '32,5', '12', '9', '36,5',
       '51', '55,5', '40,5', '44', '1', '61', '56', '50', '36', '45,5',
       '4435', '20,5', '62', '41', '52', '63', '23,5', '0,92', '43', '60',
       '39', '10', '64', '13', '48', '0,75', '-3', '-12', '53', '57',
       '80', '250', '70', '24,5', '6', '0,67', '30,5', '0,42', '34,5',
       '74'], dtype=object)

Pobieżna analiza powyższej tablicy wykazuje różne błędy zawarte w wartościach tej kolumny:
* występują wartości błędne np. .9 - brak możliwości ustalenia, czy chodziło o 9 czy 0.9
* występują wartości niemożliwe np. -12, -3, 250, 44435  

Ciężko analizować każdą pojedynczą wartość oddzielnie, a ustalenie poprawnej wartości w w/w przypadkach nie jest możliwe, dlatego postanowiono zastąpić wartości błędne oraz nienależące do przedziału [0,130] wartością `nan`

In [296]:
for i, row in titanicOutput.iterrows():
    try:
        if float(row["Age"]) < 0 or float(row["Age"]) > 130:
            titanicOutput.at[i, "Age"] = np.NaN
        else:
            titanicOutput.at[i, "Age"] = float(row["Age"])
    except ValueError:
        titanicOutput.at[i, "Age"] = np.NaN

titanicOutput["Age"].unique()

array([22.0, 38.0, 26.0, 35.0, nan, 54.0, 2.0, 27.0, 14.0, 4.0, 58.0,
       20.0, 55.0, 31.0, 34.0, 15.0, 8.0, 19.0, 40.0, 0.9, 66.0, 28.0,
       42.0, 21.0, 18.0, 3.0, 7.0, 49.0, 29.0, 65.0, 5.0, 11.0, 45.0,
       17.0, 32.0, 16.0, 25.0, 30.0, 33.0, 23.0, 24.0, 46.0, 59.0, 71.0,
       37.0, 47.0, 12.0, 9.0, 51.0, 44.0, 1.0, 61.0, 56.0, 50.0, 36.0,
       62.0, 41.0, 52.0, 63.0, 43.0, 60.0, 39.0, 10.0, 64.0, 13.0, 48.0,
       53.0, 57.0, 80.0, 70.0, 6.0, 74.0], dtype=object)

W chwili obecnej wszystkie wartości tej kolumny są już wartościami liczbowymi, więc należy zmienić typ danych na `float64`. 

In [297]:
titanicOutput[["Age"]] = titanicOutput[["Age"]].astype(float)

Po konwersji typów w chwili obecnej można wyświetlić podstawowe statystyki (m.in. wartość minimalną, maksymalną, średnią, kwartyle) związane z kolumną:

In [298]:
titanicOutput["Age"].describe()

count    690.000000
mean      29.743333
std       14.412508
min        0.900000
25%       20.000000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

Można wyróżnić wiele sposobów uzupełniania brakujących wartości w kolumnach. Jednym z nich jest wypełnianie ich średnią istniejących wartości. Wartość średnia w kolumnie wynosi ok. 30 i tą wartością uzupełniono brakujące pola w analizowanej kolumnie. Na tym analizę zakończono.

In [299]:
titanicOutput["Age"] = titanicOutput["Age"].fillna(round(titanicOutput["Age"].mean()))

## 12. Analiza kolumny `Fare`
Kolumna `Fare`, podobnie jak kolumna `Age` została zinterpretowana jako typ `object`, co nie jest wskazaną sytuacją. Przeanalizujmy występujące w niej wartości.

In [300]:
titanic["Fare"].unique()

array(['7,25', '71,2833', '7,925', '53,1', '8,05', '8,4583', '51,8625',
       '21,075', '11,1333', '30,0708', '16,7', '26,55', '7,8542', '16',
       '29,125', '13', '18', '7,225', '26', '8,0292', '31,3875', '263',
       '7,8792', '7,8958', '27,7208', '146,5208', '7,75', '10,5',
       '82,1708', '52', '7,2292', '11,2417', '9,475', '21', '41,5792',
       '15,5', '21,6792', '17,8', '39,6875', '7,8', '76,7292', '61,9792',
       '35,5', '27,75', '46,9', '80', '83,475', '27,9', '15,2458',
       '8,1583', '8,6625', '73,5', '14,4542', '56,4958', '7,65', '29',
       '12,475', '9', '9,5', '7,7875', '47,1', '15,85', '34,375',
       '61,175', '20,575', '34,6542', '63,3583', '23', '77,2875',
       '8,6542', '7,775', '24,15', '9,825', '14,4583', '247,5208',
       '31,275', '7,1417', '22,3583', '6,975', '7,05', '14,5', '15,0458',
       '26,2833', '9,2167', '79,2', '6,75', '11,5', '36,75', '7,7958',
       '12,525', '66,6', '7,3125', '61,3792', '7,7333', '69,55', '16,1',
       '15,75', '2

Ponownie mamy do czynienia z wartościami niemożliwymi (-90, 0) - zastąpmy je oraz ew. błędne elementy wartością `nan`. Dodatkowym problemem występującym wśród zawartości tej kolumny jest liczba miejsc po przecinku - waluty powinny mieć zawsze 2 miejsca po przecinku. Dostosowano wartości i zmieniono typ danych na liczby rzeczywiste.

In [301]:
titanicOutput["Fare"] = pd.to_numeric(titanicOutput["Fare"].astype(str).str.replace(',','.'), errors='coerce').round(2)
for i, row in titanicOutput.iterrows():
    if row["Fare"] <= 0:
            titanicOutput.at[i, "Fare"] = np.NaN

titanicOutput["Fare"].unique()

array([  7.25,  71.28,   7.92,  53.1 ,   8.05,   8.46,  51.86,  21.08,
        11.13,  30.07,  16.7 ,  26.55,   7.85,  16.  ,  29.12,  13.  ,
        18.  ,   7.22,  26.  ,   8.03,  31.39, 263.  ,   7.88,   7.9 ,
        27.72, 146.52,   7.75,  10.5 ,  82.17,  52.  ,   7.23,  11.24,
         9.48,  21.  ,  41.58,  15.5 ,  21.68,  17.8 ,  39.69,   7.8 ,
        76.73,  61.98,  35.5 ,  27.75,  46.9 ,  80.  ,  83.48,  27.9 ,
        15.25,   8.16,   8.66,  73.5 ,  14.45,  56.5 ,   7.65,  29.  ,
        12.48,   9.  ,   9.5 ,   7.79,  47.1 ,  15.85,  34.38,  61.18,
        20.58,  34.65,  63.36,  23.  ,  77.29,   8.65,   7.78,  24.15,
         9.82,  14.46, 247.52,  31.28,   7.14,  22.36,   6.98,   7.05,
        14.5 ,  15.05,  26.28,   9.22,  79.2 ,   6.75,  11.5 ,  36.75,
        12.52,  66.6 ,   7.31,  61.38,   7.73,  69.55,  16.1 ,  15.75,
        20.52,  55.  ,  25.92,  33.5 ,  30.7 ,  25.47,  28.71,    nan,
        39.  ,  22.02,  50.  ,   8.4 ,   6.5 ,  10.46,  18.79,  31.  ,
      

Obliczam wartość średnią:

In [302]:
titanicOutput["Fare"].mean()

32.782182857142814

Wartość średnia w kolumnie wynosi ok. 33 i tą wartością uzupełniono brakujące pola w analizowanej kolumnie. Na tym analizę zakończono.

In [303]:
titanicOutput["Fare"] = titanicOutput["Fare"].fillna(round(titanicOutput["Fare"].mean()))

## 13. Analiza pozostałych kolumn
Kolumna `Ticket` nie została zmodyfikowana ze względu na różnorodny typ danych zawarty w jej wartościach, natomiast kolumna `Name` zostanie wykorzystana jako jedna z kolumn wyłącznie w celu usunięcia duplikatów.  
## 14. Usunięcie duplikatów
Na zakończenie analizy postanowiono usunąć ewentualne duplikaty danych. W tym celu sprawdzono, ile ich jest na podstawie kolumn `Name` oraz `Ticket`:

In [304]:
duplicates = filter(None, titanicOutput.duplicated(subset=["Name", "Ticket"]))
duplicateCount = 0
for element in duplicates:
    duplicateCount = duplicateCount + 1
    
print('Duplikatów: ')
print(duplicateCount)

Duplikatów: 
3


Ze względu na fakt, że znaleziono duplikaty, postanowiono je usunąć:

In [305]:
titanicOutput = titanicOutput.drop_duplicates(subset=["Name", "Ticket"])
titanicOutput.shape[0]

889

Liczba rekordów się zmniejszyła, co potwierdza poprawność wykonanej operacji.

## 15. Zapis wynikowego pliku
Na zakończenie zapisano wynikowy plik na dysku twardym komputera.

In [307]:
titanicOutput.to_csv('TitanicCleaned.tsv', sep = '\t')

## 16. Podsumowanie
Korzystając z różnych funkcji biblioteki `Pandas` przygotowano dane do analizy poprzez ich wyczyszczenie, uzupełnienie brakujących wartości oraz usunięcie wszelkich błędnych lub nadmiarowych danych. Dzięki temu analiza danych będzie możliwa i dużo prostsza niż w przypadku oryginalnego zbioru danych.